##### [< Forrige](8%20-%20git%2C%20pythonfiler%20og%20IDE.ipynb)     |     [Neste >](10%20-%20statsmodels.ipynb)

# 9 - webskraping med python

Når vi skal skrape nettsider, analyserer vi "kildekoden" som ligger bak nettsiden. I de fleste nettleser kan du enkelt se kildekoden ved å høyreklikke på siden og velge "view page source" eller lignende. I denne leksjonen skal vi begynne med å skrape rentebarometeret til Norsk Famileøkonomi. For å se hva vi skal skrape kan du derfor gå til [https://www.norskfamilie.no/barometre/rentebarometer/](https://www.norskfamilie.no/barometre/rentebarometer/), høyreklikke og velge å se kildekoden. 

Elementer er markert i en nettside med såkalte "html-tagger". For eksempel lager du kursiv på en nettside ved å skrive `<i>kursiv</i>`. Denne teksten er skrevet i "markdown", som også forstår html-tagger. <i>Om du leser dette interaktivt i en jupyterfil kan du dobbelklikke her og se at denne setningen er skrevet inne i kursivtagger</i>.

Når vi skraper websider er innholdet vi er interessert i veldig ofte inne i en tabell. Det er det her også. Gjør et tekstsøk i kildekoden etter "\<table". Det finnes kun én plass i dokumentet, og markerer begynnelsen på tabellen. Søker du én gang til med "\</table\>" finner du hvor tabellen ender. 

I mellom disse taggene er det en god del kode som kanskje ser veldig komplisert ut. Men vi trenger kun å forholde oss til følgende tre typer tagger:

* `<tr>`: rad
* `<th>`: overskrift
* `<td>`: celle

For å hente ut innholdet i tabellen må vi altså søke etter disse taggene, etter at vi har identifisere teksten mellom "tabell"-taggene. Det finnes heldigvis et veldig godt verktøy for dette i python, som heter `BeutifulSoup` (`pip install beautifulsoup4` i kommandovinduet om det ikke er installert). 

Med dette verktøyet kan du enkelt finne de taggene du ønsker. Vi starter med å finne selve tabellen, etter å ha bruke pakken `requests` til å laste ned html-filen:

In [1]:
from bs4 import BeautifulSoup
import requests

def fetch_html_tables(url):
    "Returns a list of tables in the html of url"
    page = requests.get(url)
    bs=BeautifulSoup(page.content)
    tables=bs.find_all('table')
    return tables

tables=fetch_html_tables('https://www.norskfamilie.no/barometre/rentebarometer/')
table_html=tables[0]

#printing top
print(str(table_html)[:1000])

<table class="table table-striped table-hover barometer">
<thead>
<tr>
<th> </th>
<th>Bank</th>
<th> </th>
<th class="d-none d-sm-table-cell">Navn</th>
<th>Nominell</th>
<th class="d-none d-sm-table-cell">Sikkerhets<br/>gebyr</th>
<th class="d-none d-sm-table-cell">Etablerings<br/>gebyr</th>
<th class="d-none d-sm-table-cell">Termin</th>
<th>Effektiv</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td>SpareBank 1 Nordmøre</td>
<td>
<button class="popover_info btn btn-none" data-html="true" data-placement="left" data-toggle="tooltip" title="Nominell: &lt;strong&gt;1,35&lt;/strong&gt;&lt;br&gt;Sikkerhetsgebyr: &lt;strong&gt;1 200&lt;/strong&gt;&lt;br&gt;Etableringsgebyr: &lt;strong&gt;0&lt;/strong&gt;&lt;br&gt;Termingebyr: &lt;strong&gt;75&lt;/strong&gt;&lt;br&gt;Effektiv rente: &lt;strong&gt;1,45&lt;/strong&gt;" type="button">
<i class="fa fa-info-circle"></i>
</button>
</td>
<td class="d-none d-sm-table-cell">Grønt førstehjemslån</td>
<td>1,35</td>
<td class="d-none d-sm-table-cell">1 200<

Det vi får ut med `bs.find_all('table')` er altså en liste med alle partier i teksten med matchende `<table>`-`</table>`-tagger. I dette dokumentet er det bare én tabell, så listen har bare ett element. Vi må nå søke videre inne i tabellen etter innholdstaggene. Vi bruker samme funksjon til det. Her er to funksjoner som sammen finner innholdstaggene og returnerer en tabell:

In [2]:
def html_to_table(html):
    "Returns the table defined in html as a list"
    #defining the table:
    table=[]
    #iterating over all rows
    for row in html.find_all('tr'):
        r=[]
        #finding all cells in each row:
        cells=row.find_all('td')
        
        #if no cells are found, look for headings
        if len(cells)==0:
            cells=row.find_all('th')
            
        #iterate over cells:
        for cell in cells:
            cell=format(cell)
            r.append(cell)
        
        #append the row to t:
        table.append(r)
    return table

def format(cell):
    "Returns a string after converting bs4 object cell to clean text"
    if cell.content is None:
        s=cell.text
    elif len(cell.content)==0:
        return ''
    else:
        s=' '.join([str(c) for c in cell.content])
        
    #here you can add additional characters/strings you want to 
    #remove, change punctuations or format the string in other
    #ways:
    s=s.replace('\xa0','')
    s=s.replace('\n','')
    return s

table=html_to_table(table_html)

#printing top
print(str(table)[:1000])

[['', 'Bank', '', 'Navn', 'Nominell', 'Sikkerhetsgebyr', 'Etableringsgebyr', 'Termin', 'Effektiv'], ['1', 'SpareBank 1 Nordmøre', '', 'Grønt førstehjemslån', '1,35', '1200', '0', '75', '1,45'], ['2', 'Landkreditt Bank AS', '', 'Grønt Boliglån', '1,55', '1000', '0', '0', '1,56'], ['3', 'Statens pensjonskasse', '', 'Boliglån inntil 80 %', '1,49', '0', '0', '50', '1,56'], ['4', 'Etne Sparebank', '', '"Him te Etne" - lånet', '1,55', '0', '0', '50', '1,62'], ['5', 'SpareBank 1 SMN', '', 'Grønt førstehjemslån', '1,60', '1200', '0', '60', '1,68'], ['6', 'SpareBank 1 Nordmøre', '', 'Grønt boliglån', '1,60', '1200', '0', '75', '1,70'], ['7', 'Høland og Setskog Sparebank', '', 'Grønt Boliglån spesial', '1,64', '0', '0', '60', '1,73'], ['8', 'Orkla Sparebank', '', 'Grønt boliglån', '1,65', '1000', '0', '60', '1,74'], ['9', 'KLP Banken AS', '', 'Grønt boliglån medlem', '1,70', '0', '0', '35', '1,76'], ['10', 'Landkreditt Bank AS', '', 'Boliglån 50%', '1,75', '500', '0', '0', '1,76'], ['11', 'Oslo 

Den første funksjonen itererer over tabellceller, mens den andre funksjonen konverterer innholdet fra et bs4-objekt med html-kode til leselig tekst. 

Vi har nå skrapet siden, og hentet ut tabellen. For å gjøre den mer leselig, kan vi lagre den som en fil. Når vi lager filer i python bruker vi den innebygde `open`-funksjonen. Om vi kaller filen for "rentebarometer.csv", kan vi opprette filen ved å kjøre `f=open('rentebarometer.csv','w')`. Strengen `'w'` betyr at vi åpner filen for skriving (*writing*, i motsetning til lesing/*reading* markert med `'r'`. Vi fyller filen med innhold med `f.write()`. 

For å skille kolonnene skal vi her bruke semikolon ';'. Python har en enkel måte å konvertere en liste til en streng med skilletegn. En tar utgangspunkt i skilletegnet, og bruker metoden `join()` på det. For eksempel: 

In [3]:
';'.join(table[0])

';Bank;;Navn;Nominell;Sikkerhetsgebyr;Etableringsgebyr;Termin;Effektiv'

Vi kan nå åpne filen for skriving og iterere over rader og skrive dem til filen. 

In [4]:
def save_data(file_name,table):
    "Saves table to file_name"
    f=open(file_name,'w')
    for row in table:
        f.write(';'.join(row)+'\n')
    f.close()
    
save_data('rentebarometer.csv',table)

Vi kan ta en kikk på dataene med Pandas (`encoding='latin1'` er for å få med æ,ø,å):

In [5]:
import pandas as pd
pd.read_csv('rentebarometer.csv', delimiter=';', encoding='latin1')

,Unnamed: 0,Bank,Unnamed: 2,Navn,Nominell,Sikkerhetsgebyr,Etableringsgebyr,Termin,Effektiv
0,1,SpareBank 1 Nordmøre,NaN,Grønt førstehjemslån,"1,35",1200,0,75,"1,45"
1,2,Landkreditt Bank AS,NaN,Grønt Boliglån,"1,55",1000,0,0,"1,56"
2,3,Statens pensjonskasse,NaN,Boliglån inntil 80 %,"1,49",0,0,50,"1,56"
3,4,Etne Sparebank,NaN,Him te Etne - lånet,"1,55",0,0,50,"1,62"
4,5,SpareBank 1 SMN,NaN,Grønt førstehjemslån,"1,60",1200,0,60,"1,68"
...,...,...,...,...,...,...,...,...,...
335,336,Nordax Bank AB (publ),NaN,Nordax Boliglån,"5,99",0,10000,35,"6,20"
336,337,Danske Bank,NaN,Toppfinansiering,"6,20",1000,0,45,"6,44"
337,338,INSTABANK ASA,NaN,Lån med sikkerhet,"6,70",0,5000,50,"6,97"
338,339,Kraft Bank ASA,NaN,Refinansieringslån med sikkerhet,"6,95",0,0,75,"7,27"


La oss sette sammen dette til én kode, som du kan lagre i en fil om du vil:

In [6]:
from bs4 import BeautifulSoup
import requests

def scrape(url, file_name):
    table=[]
    tables=fetch_html_tables(url)
    #iterate over all tables, if there are more than one:
    for tbl in tables:
        #exends table so that table is a list containing elements 
        #from all tables:
        table.extend(html_to_table(tbl))
    #saving it:
    save_data(file_name,table)
    return table
 

def save_data(file_name,table):
    "Saves table to file_name"
    f=open(file_name,'w')
    for row in table:
        f.write(';'.join(row)+'\n')
    f.close()


def fetch_html_tables(url):
    "Returns a list of tables in the html of url"
    page = requests.get(url)
    bs=BeautifulSoup(page.content)
    tables=bs.find_all('table')
    return tables

def html_to_table(html):
    "Returns the table defined in html as a list"
    #defining the table:
    table=[]
    #iterating over all rows
    for row in html.find_all('tr'):
        r=[]
        #finding all cells in each row:
        cells=row.find_all('td')
        
        #if no cells are found, look for headings
        if len(cells)==0:
            cells=row.find_all('th')
            
        #iterate over cells:
        for cell in cells:
            cell=format(cell)
            r.append(cell)
        
        #append the row to t:
        table.append(r)
    return table

def format(cell):
    "Returns a string after converting bs4 object cell to clean text"
    if cell.content is None:
        s=cell.text
    elif len(cell.content)==0:
        return ''
    else:
        s=' '.join([str(c) for c in cell.content])
        
    #here you can add additional characters/strings you want to 
    #remove, change punctuations or format the string in other
    #ways:
    s=s.replace('\xa0','')
    s=s.replace('\n','')
    return s

Med denne koden kan vi nå skrape hvilken som helst nettside med tabeller vi ønsker å få tak i. For eksempel om vi ønsker å hente timeplanen til kurset:

In [7]:
url='https://timeplan.uit.no/emne_timeplan.php?sem=22v&module[]=SOK-1005-1'
file_name='schedule.csv'

table=scrape(url,file_name)

s='\n'.join(['\t'.join(row) for row in table])


#printing top
print(str(s)[:1000])

Uke  2	Mandag  10.01.2022	Tirsdag  11.01.2022	Onsdag  12.01.2022	Torsdag  13.01.2022	Fredag  14.01.2022
08:00					
09:00					
10:00					
11:00					
12:00					
13:00					
14:00				Aktiviteter i tidsrommet 14:15-16:0014:15-16:00VIRTUELT _Digital undervisningSOK-1005-1ForelesningØ. Myrland	14:15-16:00	VIRTUELT _Digital undervisning	SOK-1005-1	Forelesning	Ø. Myrland		
14:15-16:00	VIRTUELT _Digital undervisning
SOK-1005-1	Forelesning
Ø. Myrland	
15:00				
16:00					
14:15-16:00	VIRTUELT _Digital undervisning
SOK-1005-1	Forelesning
Ø. Myrland	
Uke  3	Mandag  17.01.2022	Tirsdag  18.01.2022	Onsdag  19.01.2022	Torsdag  20.01.2022	Fredag  21.01.2022
08:00					
09:00					
10:00					
11:00					
12:00					
13:00					
14:00				Aktiviteter i tidsrommet 14:15-16:0014:15-16:00VIRTUELT _Digital undervisningSOK-1005-1ForelesningØ. Myrland	14:15-16:00	VIRTUELT _Digital undervisning	SOK-1005-1	Forelesning	Ø. Myrland		
14:15-16:00	VIRTUELT _Digital undervisning
SOK-1005-1	Forelesning
Ø. Myrland	
15:00				
1

#### Øvelse:

Resultatet vises ved å lage en tabluatorseparert streng fordi filen ikke kan åpnes med pandas. Grunnen til det er at radene her her ulikt antall kolonner. Vi ser også at det trengs litt arbeid her for å organisere dataene bedre. Det kan du gjøre ved å redigere i funksjonene `html_to_table` og `format`. 

## Når nettsiden ikke er "vennligsinnet"

Det er ikke alle nettsideeiere som synes det er greit at vi skraper nettsidene deres. For ordens skyld så er det altså helt lovlig å skrape nettsider. Når noen legger ut data på en nettside har de offentliggjort dataene, og kan ikke bestemme hvordan dataene skal leses. Dette gjelder selv om de legger ut beskjed om noe annet. 

Det som kanskje *kan* være ulovlig, er å videreformidle dataene i kommersiell. Når nettsiden er vanskelig å skrape, er *selenium* av google en veldig nyttig pakke. Med den kan koden din opptre som en vanlig bruker. Så lenge du kan se dataene på skjermen din, bør det da i prinsippet være mulig å skrape enhver side. 

Vi har ikke tid til å gå inn på bruken av selenium i dette kurset, men her er en kode som bruker selenium til å skrape [nordpool.no](https://www.nordpoolgroup.com/Market-data1/Dayahead/Area-Prices/NO/Monthly/?view=table). De er ikke spesielt interessert i at vi gjør det, om du forsøker å skrape flere ganger kommer det opp en advarsel om at det er ulovlig, som altså ikke medfører riktighet. 

Her er imidlertid en kode som gjøre det mulig å skrape Nordpools nettsider med selenium (må kjøres som en fil):


In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

def scrape(url):
    #get the html from the url
    html=get_page(url)

    #read it with BS
    bs=BeautifulSoup(html)

    #extract all tables and put in array t
    tables=bs.find_all('table')
    t=[]
    for tbl in tables:
        t.extend(html_to_table(tbl))


    #save the result:
    f=open('table.csv','w')
    for row in t:
        f.write(';'.join(row)+'\n')
    f.close()
    a=0

def get_page(url):
    """returns a specific part of the web site. For some reason 
    BS cannot easily extract the second table, which is the one with data
    which we are interested in"""
    #open the url in browser
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)

    #sometimes the page is not loaded properly, so repeating until we have fetched a 
    #postive length string:
    for i in range(1000):
        s=find_string_between(driver.page_source, '<table id="datatable">','</tfoot></table>')
        if len(s)>0:
            break
        time.sleep(1)
    return s	

def html_to_table(tbl):
    """Extracts the table from a table found with BS"""

    #initiates the list object that will be returned:
    a=[]
    #iterates over all table rows:
    for row in tbl.find_all('tr'):
        #initiates the current row to be added to a:
        r=[]

        #identifies all cells in row:
        cells=row.find_all('td')
        #if there were no normal cells, there might be header cells:
        if len(cells)==0:
            cells=row.find_all('th')

        #iterate over cells 
        for cell in cells:
            cell=format(cell)
            r.append(cell)
        a.append(r)
    return a


def format(cell):
    """Returns the text of cell"""

    if cell.content is None:
        return cell.text
    if len(cell.content)==0:
        return ''
    s=''
    s=' '.join([str(c) for c in cell.content])

    #if there is unwanted contents, replace it here:
    s=s.replace('\xa0','')
    s=s.replace('\n','')
    return s

def find_string_between(string,a,b):
    "returns the substring of string between expressions a and b" 
    a=string.find(a)
    b=string.find(b)+len(b)
    return string[a:b]

#scrape('https://www.nordpoolgroup.com/Market-data1/Dayahead/Area-Prices/NO/Monthly/?view=table')

##### [< Forrige](8%20-%20git%2C%20pythonfiler%20og%20IDE.ipynb)     |     [Neste >](10%20-%20statsmodels.ipynb)